# **Ayudantía 10 - Ejercicios C2**
## Etienne Rojas

Ante cualquier duda o posible corrección, por favor mandar un correo a `etienne.rojas@sansano.usm.cl` 


---

In [4]:
import numpy as np
import matplotlib.pyplot as plt

# **Ejercicio 2.4 Ajedrez**

## Función Auxiliar

In [ ]:
def ChessMatrixTransform(n, Dn):
    '''
    Transforma la matriz compacta D^{(n)} [2N X N+1] en la matriz C^{(n)} [2N X 2N]
    La matriz C^{(n)} es una matriz de ajedrez que contiene los elementos de D^{(n)}
    En posiciones espaciadas por elementos nulos a excepcion de las ultimas 2 columnas
    que son iguales a las ultimas 2 columnas de D^{(n)}.

    Args:
        n (int): Índice de la matriz C^{(n)} y D^{(n)}.
        Dn (ndarray): Matriz compacta de tamaño [2N X N+1] que almacena los no nulos de C^{(n)}.
    
    Returns:
        C (ndarray): Matriz de ajedrez de tamaño [2N X 2N] que contiene los elementos de D^{(n)} en forma de ajedrez.
    '''
    # PASO 1: CALCULAMOS EL TAMAÑO DE LA MATRIZ C^{(n)}
    filas = 2 * n
    columnas = 2 * n

    # PASO 2: INICIALIZAMOS LA MATRIZ C^{(n)} CON CEROS
    C = np.zeros((filas, columnas))

    # PASO 3: SABEMOS QUE LAS ULTIMAS 2 COLUMNAS DE C^{(n)} = D^{(n)}
    C[:, -2:] = Dn[:, -2:]

    # PASO 4: GENERAMOS LOS ÍNDICES PARA LAS FILAS Y COLUMNAS DE LA MATRIZ DE AJEDREZ
    indices_pares_fila = np.arange(filas)[::2]   
    indices_impares_fila = np.arange(filas)[1::2]
    columnas_ajedrez_pares = np.arange(0, 2 * (n - 1), 2)
    columnas_ajedrez_impares = np.arange(1, 2 * (n - 1), 2)

    # PASO 5: ASIGNAMOS LOS VALORES DE D^{(n)} A LAS POSICIONES CORRESPONDIENTES EN C^{(n)}
    for i in indices_pares_fila:
        C[i, columnas_ajedrez_pares] = Dn[i, :n-1]

    for i in indices_impares_fila:
        C[i, columnas_ajedrez_impares] = Dn[i, :n-1]

    return C

n = 3
Dn = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 1, 2],
    [3, 4, 5, 6],
    [7, 8, 9, 10],
    [1, 2, 3, 4],
])
C = ChessMatrixTransform(n, Dn)
print(C)

[[ 1.  0.  2.  0.  3.  4.]
 [ 0.  5.  0.  6.  7.  8.]
 [ 9.  0. 10.  0.  1.  2.]
 [ 0.  3.  0.  4.  5.  6.]
 [ 7.  0.  8.  0.  9. 10.]
 [ 0.  1.  0.  2.  3.  4.]]


In [7]:
def OneStepJacobi(n, Dn, b, x0):
    '''
    Realiza una iteración del método de Jacobi para la matriz C^{(n)} almacenada en Dn.
    
    Args:
        n (int): Índice de la matriz C^{(n)} y D^{(n)}.
        Dn (ndarray): Matriz compacta de tamaño 2n x (n+1) que almacena los no nulos de C^{(n)}.
        b (ndarray): Vector del lado derecho de tamaño 2n.
        x0 (ndarray): Vector inicial de tamaño 2n.
    
    Returns:
        x1 (ndarray): Nuevo vector después de una iteración de Jacobi.
    '''
    # PASO 1: reconstruir la matriz C
    C = ChessMatrixTransform(n, Dn)
    
    # PASO 2: Seprar C en L,D^{-1},U 
    C_diag_inv = np.diag(1 / np.diag(C))
    C_low = np.tril(C, k=-1)
    C_up = np.triu(C, k=1)
    
    
    # PASO 4: aplicar Jacobi
    x1 = C_diag_inv @ (b - (C_low + C_up) @ x0)
    
    return x1

## **Testear el Código**

In [8]:
n = 2
Dn = np.array([
    [50.0, 1.0, 1.0],
    [50.0, 1.0, 1.0],
    [1.0, 50.0, 1.0],
    [1.0, 1.0, 50.0],
])

C = ChessMatrixTransform(n, Dn)

x_real = np.array([1.0, 2.0, 3.0, 4.0])

b = C @ x_real

x0 = np.array([10.0, 10.0, 10.0, 10.0])


x = x0.copy()
for i in range(7):
    x = OneStepJacobi(n, Dn, b, x)
    print(f"Iteración {i+1}: {x}")


print("\nSolución real:", x_real)
print("Aproximación final:", x)
print("Error absoluto:", np.abs(x - x_real))

Iteración 1: [0.74 1.74 2.7  3.7 ]
Iteración 2: [1.012  2.012  3.0112 4.0112]
Iteración 3: [0.999552 1.999552 2.999536 3.999536]
Iteración 4: [1.00001856 2.00001856 3.00001824 4.00001824]
Iteración 5: [0.99999927 1.99999927 2.99999926 3.99999926]
Iteración 6: [1.00000003 2.00000003 3.00000003 4.00000003]
Iteración 7: [1. 2. 3. 4.]

Solución real: [1. 2. 3. 4.]
Aproximación final: [1. 2. 3. 4.]
Error absoluto: [1.17247989e-09 1.17247989e-09 1.17503962e-09 1.17503962e-09]


---

## **Ejercicio 2.12 ¿Es o no es tridiagonal?**

#### **Parámetros Generales**

In [57]:
a = 10
b = -4
c = 1/2
n = 3

In [71]:
def build_matrix_A(a,b,c,n):
    """
    Construye la matriz A de un sistema tridiagonal de tamaño n x n.
    
    Input:
        a (float): Valor en la diagonal secundario
        b (float): Valor en la diagonal secundaria superior.
        c (float): Valor en la diagonal secundaria inferior.
        n (int): Tamaño de la matriz.
    
    Output:
        A (ndarray): Matriz tridiagonal de tamaño n x n.
    """
    A = np.zeros((n, n))

    np.fill_diagonal(A[1:, :-1], b)
    np.fill_diagonal(A, a)
    np.fill_diagonal(A[:-1, 1:], c)
    
    A_invertida = A[::-1]
    A = A_invertida
    return A

A = build_matrix_A(a, b, c, n)
print("Matriz A:")
print(A)

Matriz A:
[[ 0.  -4.  10. ]
 [-4.  10.   0.5]
 [10.   0.5  0. ]]


In [61]:
def build_vector_b(n):
    """
    Construye el vector b de la ecuacion Ax=b
    Donde los elementos van desde son = 1/i hasta 1/n
    con i = 1, 2, ..., n.
    
    Input:
        n (int): Tamaño del vector.
    
    Output:
        b (ndarray): Vector de tamaño n con los elementos 1/i.
    """
    b = (1) / (np.arange(1, n + 1))
    return b

vector_b = build_vector_b(n)
print("Vector b:")
print(vector_b)

Vector b:
[1.         0.5        0.33333333]


In [ ]:
def solve_system(A, vector_b,n):
    """
    Resuelve el sistema Ax = b utilizando el método desarrollado.
    En la parte teórica del problema.
    Input:
        A (ndarray): Matriz del sistema.
        b (ndarray): Vector del lado derecho.
        n (int): tamaño del vector b y de la matriz A.
    
    Output:
        x (ndarray): Solución del sistema.
    """

    A = A.copy()
    A = A[::-1]
    vector_b = vector_b.copy()
    vector_b = vector_b[::-1]

    # PASO 1: Extraer diagonales de la matriz tridiagonal
    main_diag = np.diagonal(A).copy()
    upper_diag = np.diagonal(A, offset=1).copy()
    lower_diag = np.diagonal(A, offset=-1).copy()
    # print("Diagonal principal:", main_diag)
    # print("Diagonal superior:", upper_diag)
    # print("Diagonal inferior:", lower_diag)

    # PASO 2: OPERACIONES DE ELIMINACIÓN
    elimination_factors = np.zeros(n-1)
    new_b_vector = np.zeros(n)
    
    # Inicializar primera fila 1
    elimination_factors[0] = upper_diag[0] / main_diag[0]  # c / a
    new_b_vector[0] = vector_b[0] / main_diag[0]           # b[0] / a
    
    # Eliminación para filas 2 a n-1
    for i in range(1, n-1):
        denom = main_diag[i] - lower_diag[i-1] * elimination_factors[i-1]
        elimination_factors[i] = upper_diag[i] / denom
        new_b_vector[i] = (vector_b[i] - lower_diag[i-1] * new_b_vector[i-1]) / denom
    
    # Eliminación ultima fila n
    denom = main_diag[-1] - lower_diag[-1] * elimination_factors[-1]
    new_b_vector[-1] = (vector_b[-1] - lower_diag[-1] * new_b_vector[-2]) / denom
    
    # PASO 3: Sustitución hacia atrás
    x = np.zeros(n)
    x[-1] = new_b_vector[-1]
    for i in range(n-2, -1, -1):
        x[i] = new_b_vector[i] - elimination_factors[i] * x[i+1]
    
    return x


print("Resolviendo el sistema Ax = b...")
print(A,"x=", vector_b)

print("Solución del sistema:")
x_solution = solve_system(A, vector_b, n)
print("x:",x_solution)

print("Comprobación Ax = b:")
Ax = A @ x_solution
print("Ax:", Ax)
print("b:", vector_b)



Resolviendo el sistema Ax = b...
[[ 0.  -4.  10. ]
 [-4.  10.   0.5]
 [10.   0.5  0. ]] x= [1.         0.5        0.33333333]
Solución del sistema:
Diagonal principal: [10. 10. 10.]
Diagonal superior: [0.5 0.5]
Diagonal inferior: [-4. -4.]
x: [0.03052885 0.05608974 0.1224359 ]
Comprobación Ax = b:
Ax: [1.         0.5        0.33333333]
b: [1.         0.5        0.33333333]
